In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
import numpy as np
import pandas as pd

In [ ]:
class MysteryImageDataset(Dataset):
    def __init__(self, csv_file, is_test=False):
        self.is_test = is_test
        df = pd.read_csv(csv_file)
        
        self.ids = df.iloc[:, 0].values
        
        if self.is_test:
            self.X = df.iloc[:, 1:].values.astype(np.float32)
            self.y = None
        else:
            self.X = df.iloc[:, 1:-1].values.astype(np.float32)
            self.y = df.iloc[:, -1].values.astype(np.int64) 

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        features_1d = self.X[idx] # Shape (205,)
        
        features_padded = np.pad(features_1d, (0, 20), 'constant', constant_values=0)
        image = torch.tensor(features_padded).view(1, 15, 15)
        
        if self.is_test:
            return image
        else:
            label = torch.tensor(self.y[idx])
            return image, label

In [46]:
# import data from files
full_dataset = MysteryImageDataset("data/train.csv", is_test=False)
submission_dataset = MysteryImageDataset("data/test.csv", is_test=True)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_data, val_data = random_split(full_dataset, [train_size, val_size])


In [47]:
# create data loaders
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(submission_dataset, batch_size=batch_size, shuffle=False) 

In [48]:
input_dim = 205

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.feature_extractor = nn.Sequential(
        # BLOCK 1: FINDING EDGES
        # (64, 1, 15, 15) -> One grayscale image
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1), 
        # Shape stays (16, 15, 15) because of padding=1 and 3x3 kernel shrinking.
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2), 
        # New Shape: (16, 7, 7)
        # BLOCK 2: FINDING SHAPES
        # (16, 7, 7)
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1), 
        # Shape stays (32, 7, 7)
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        # New Shape: (32, 3, 3)
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1), 
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2) 
        # Final Shape: (64, 1, 1)
)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            
            nn.Linear(64, 512), 
            nn.ReLU(),
            nn.Linear(512, 256), 
            nn.ReLU(),
            nn.Linear(256, 5) # 5 Classes
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

In [50]:
device = torch.cuda.current_device().type if torch.cuda.is_available() else "cpu"
model = SimpleCNN().to(device)
learning_rate = 1e-3    
batch_size = 64
epochs = 5

In [ ]:
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(logits, 1)
        correct += (preds == labels).sum().item()
    return running_loss / len(dataloader.dataset), correct / len(dataloader.dataset)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(logits, 1)
            correct += (preds == labels).sum().item()
    return running_loss / len(dataloader.dataset), correct / len(dataloader.dataset)

def test_loop(dataloader, model, output_file):
    model.eval()
    all_preds = []
    
    
    
    with torch.no_grad():

        for images in dataloader:
            images = images.to(device)
            
            
            logits = model(images)
            
            
            batch_preds = logits.argmax(1).cpu().numpy()
            all_preds.extend(batch_preds)
            
    submission_df = pd.DataFrame({
        "id": dataloader.dataset.ids, 
        "label": all_preds
    })
    

    submission_df.to_csv(output_file, index=False)
    print(f"Done! Saved {len(submission_df)} predictions to '{output_file}'")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 30
print("Starting Training...")
history_records = []
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_dataloader, loss_fn, optimizer, device)
    val_loss, val_acc = evaluate(model, val_dataloader, loss_fn, device)

    record = {
        "record_id": epoch,              
        "model_id": 5,
        "training_loss": train_loss,
        "validation_loss": val_loss,
        "training_acc": train_acc,   
        "validation_acc": val_acc
    }

    history_records.append(record)
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")
print("Done!")




Starting Training...
Epoch 1/30 | Train Loss: 1.2224 | Val Loss: 1.0532 | Val Acc: 62.62%
Epoch 2/30 | Train Loss: 0.9388 | Val Loss: 0.9228 | Val Acc: 66.81%
Epoch 3/30 | Train Loss: 0.8020 | Val Loss: 0.8360 | Val Acc: 70.50%
Epoch 4/30 | Train Loss: 0.6834 | Val Loss: 0.8136 | Val Acc: 71.56%
Epoch 5/30 | Train Loss: 0.6181 | Val Loss: 0.8030 | Val Acc: 72.75%
Epoch 6/30 | Train Loss: 0.5227 | Val Loss: 0.7786 | Val Acc: 73.12%
Epoch 7/30 | Train Loss: 0.4643 | Val Loss: 0.8119 | Val Acc: 74.19%
Epoch 8/30 | Train Loss: 0.4025 | Val Loss: 0.7942 | Val Acc: 73.31%
Epoch 9/30 | Train Loss: 0.3547 | Val Loss: 0.8948 | Val Acc: 72.38%
Epoch 10/30 | Train Loss: 0.3082 | Val Loss: 0.8409 | Val Acc: 74.38%
Epoch 11/30 | Train Loss: 0.2680 | Val Loss: 0.8121 | Val Acc: 76.00%
Epoch 12/30 | Train Loss: 0.2299 | Val Loss: 0.9345 | Val Acc: 74.50%
Epoch 13/30 | Train Loss: 0.1994 | Val Loss: 1.0257 | Val Acc: 72.44%
Epoch 14/30 | Train Loss: 0.2213 | Val Loss: 0.9504 | Val Acc: 74.38%
Epoch 15

In [ ]:

import os

output_file = "experiment_logs.csv"


new_df = pd.DataFrame(history_records)


final_df = new_df[['record_id', 'model_id', 'training_loss', 'validation_loss', 'training_acc', 'validation_acc']]


if os.path.exists(output_file):

    final_df.to_csv(output_file, mode='a', header=False, index=False)
    print(f"Appended {len(final_df)} records to existing {output_file}")
else:

    final_df.to_csv(output_file, mode='w', header=True, index=False)
    print(f"Created new file {output_file} with {len(final_df)} records")


print("\nLatest Records:")
print(final_df.head())

Appended 30 records to existing experiment_logs.csv

Latest Records:
   record_id  model_id  training_loss  validation_loss  training_acc  \
0          0         5       1.222441         1.053217      0.539687   
1          1         5       0.938844         0.922842      0.656094   
2          2         5       0.801999         0.836041      0.713906   
3          3         5       0.683378         0.813614      0.755469   
4          4         5       0.618062         0.802962      0.786250   

   validation_acc  
0        0.626250  
1        0.668125  
2        0.705000  
3        0.715625  
4        0.727500  


In [54]:
test_loop(test_dataloader, model, "submission4.csv")

Done! Saved 2000 predictions to 'submission4.csv'
